# __Lecture 09: Practical analyses in Python (continued)__

----
#### __Announcement:__ Homework 4 will be due on October 31

#### __On Tuesday, we talked about:__
- `break` and `continue` statements
- defining functions in python
- working with regular expressions using the `re` module

#### __Today we will:__
- continue practicing with the `re` module
- learn about (and practice) using the `biopython` module

###  
----

## __Review:__ Regular expressions

The `re` module offers a set of functions that allows us to search a string for a match using a __search pattern__ 

#### Here are some common elements to have in your __search pattern__:

* **Specific patterns:**
    * **Specific letter characters** (e.g., `A`, `B`, `C`, ...): Matches any occurrence of the specified letter in the string.
    * **Specific numerical digits** (e.g., `1`, `2`, `3`, ...): Matches any occurrence of the specified digit.
    * **Specific special characters** (e.g., `*`, `$`, ...): Matches any occurrence of the specified special character. Special characters need to be preceded by a `\` (e.g., `\.` for a period, `\$` for a dollar sign).
  
* **General patterns:**
    * **Numerical digits**: `\d` matches any digit (numbers `0`–`9`).
    * **Any single character**: `.` matches _any_ single character.
    * **Any letter or character from a set**: `[]` allows you to specify a set of characters to search for (e.g., `[a-n]` matches any lowercase letter from `a` to `n`).
    * **One or more occurrences**: `+` specifies one or more occurrences of the preceding pattern (e.g., `a+` matches "a", "aa", etc.).
    * **Exact number of occurrences**: `{}` specifies an exact number of occurrences of the preceding pattern (e.g., `a{6}` matches exactly six "a" characters).

* **Anchors for start/end of string**:
    * **End of string**: `$` matches the end of the string (e.g., `\d$` matches a string that ends with a digit).
    * **Beginning of string**: `^` matches the beginning of the string (e.g., `^A` matches any string that starts with "A").

* **Grouping and naming**:
    * **Grouping patterns**: `()` is used to group part of your search pattern. Everything inside the parentheses is treated as a single unit, and you can capture and reference this group.
    * **Named groups**: `(?P<name>...)` defines a named group with the specified name, allowing you to reference this part of the match later (e.g., `(?P<year>\d{4})` for a four-digit year).



#### Always remember to import the `re` module if you'd like to work with regular expressions:

In [2]:
import re

## __Review practice__: using regular expressions to parse flu subtypes

You download some strains from a database, and they have names that look like this:

In [3]:
strain = 'A/New York/3/1994 (H3N2)'

Let's build a regular expression that gets the subtype (e.g. `H3N2`) out the `strain` object

_Hints:_
- _Be sure to name the search pattern `subtype`_
- _Include the parentheses in your search pattern, but not in the final named search pattern_

In [64]:
# compile a named re for the subtype
## This searches for a pattern that consists of:
## - A capital letter
## - Followed by one or more digits
## - Then another capital letter
## - Followed by one or more digits
## The entire pattern is enclosed in parentheses.
subtype_pattern = re.compile('\((?P<subtype>[A-Z]\d+[A-Z]\d+)\)')
#subtype_pattern = re.compile('((?P<subtype>[A-Z]\d+[A-Z]\d+))')

# search for the search pattern in the string
match = subtype_pattern.search(strain)

# isolate named pattern
subtype_match = match.group('subtype')

print(subtype_match)

H3N2


----
## __Practice:__ using regular expressions to parse barcodes
Now we will use regular expressions to parse barcodes from nucleotide sequences.
For instance, you might have to do this in a single-cell RNA-seq experiment where there is a barcode at the end of each read telling you the cell that the read came from.

Imagine that our valid molecules should have sequences like this:

`CTAGCNNNNNNGATCA`

See how there is a 6-nucleotide barcode in the center of the sequence.
We have a list of sequences, and want to parse through them to figure out which ones meet the expected pattern--and get the barcode from those that do:

In [5]:
seqs = ['CTAGCatcgatGATCA',  # has barcode ATCGAT
        'CCAGCatagcaGATCA',  # does not have expected 5' sequence
        'CTAGCtacagGATCA',   # barcode too short
        'CTAGCgaccatGATCA',  # has barcode GACCAT
        'CTAGCatcgatGATCA',  # has barcode ATCGAT
        'CTAGCatcgatGGTCA',  # does not have expected 3' sequence
        ]

### __Practice part 1__
First, take a few minutes in groups to write a search pattern that will match the valid barcodes from these sequences.
When doing this, note that:

- You will want to __group__ and __name__ the barcode portion of the search pattern
- You will want to __include the fixed upstream (CTAGC) and downstream (GATCA) portions__ of the sequence in your search pattern, but not as part of the barcode subgroup

In [67]:
# find barcode in first sequence in 'seqs' list

first_seq = seqs[0].upper()

barcode_search = re.compile('CTAGC(?P<barcode>[ATCG]{6})GATCA')

match = barcode_search.search(first_seq)

barcode_match = match.group("barcode")

print(barcode_match)


ATCGAT


### __Practice part 2__

Now, modify your code to create a function that parses barcoded sequences and extracts those with valid barcodes. 
This requires setting up your function so that it does the following:

 1. Converts each sequence to uppercase (using `s.upper()`)
 2. Searches the string for a specified search pattern
 3. Determines if it matches the expected pattern (with the correct length barcode and constant upstream/downstream sequences)
 3. If it matches, extracts the barcode and add it to a dictionary to keep track of counts

A few notes to keep in mind:
- You may want to search from the start of the string (use the `^` symbol in your search pattern).
- The barcode length, upstream sequence, and downstream sequence should be passed as function arguments with default values, rather than being fixed. You’ll need to generalize your search pattern from above to accommodate these variable inputs (e.g., by adding strings and variables together).

I’ve provided the function documentation below—try to implement it.
__Take a few minutes to work through this in groups.__

In [71]:
def count_barcodes(seqs, bclen=6, upstream='CTAGC', downstream='GATCA'):
    """Parse and count barcodes.
    
    Parameters
    ----------
    seqs : list
        DNA sequences.
    bclen : int
        Length of barcode
    upstream : str
        Sequence upstream of barcode.
    downstream : str
        Sequence downstream of barcode.
        
    Returns
    -------
    dict
        Keyed by each valid barcode, value is number of times the barcode
        is observed.
        
    Note
    ----
    The function is **not** case-sensitive, and all barcodes are reported
    in upper-case.
    
    """
    
    barcode_counts = {}
    
    for seq in seqs:
        seq_upper = seq.upper()
        
        barcode_search = re.compile(upstream + '(?P<barcode>[ATCG]{' + str(bclen) + '})' + downstream)
        
        match = barcode_search.search(seq_upper)
        
        if match:
            barcode_match = match.group('barcode')
            
            if barcode_match in barcode_counts:
                barcode_counts[barcode_match] += 1
            else:
                barcode_counts[barcode_match] = 1
        
    return(barcode_counts)


Run the function once you've implemented it. Does it give the right result?

In [72]:
count_barcodes(seqs)

{'ATCGAT': 2, 'GACCAT': 1}

----
# __New module:__ Biopython

The [Biopython](https://biopython.org/) package has lots of useful functions for computational biology.

It is very handy for things like __reading in sequences__ in many different formats and __processing sequence data__: the subpackage [Bio.SeqIO](https://biopython.org/wiki/SeqIO) is your friend!

_(Do note that if you are analyzing truly large datasets, `Biopython` is not very fast and you may want to use something like [pysam](https://pysam.readthedocs.io/en/latest/api.html); but `Biopython` is a good starting point)._

First, we'll need to import the submodules `Biopython.SeqIO` and `Biopython.Seq`

In [10]:
import Bio.SeqIO
import Bio.Seq


#### __Reading in a file__
I have included the file [barcodes_R1.fastq](barcodes_R1.fastq), which has some FASTQ sequences in it.

Let's use `Biopython` to read the FASTQ entries and convert them to a list:

In [14]:
reads = Bio.SeqIO.parse('barcodes_R1.fastq', format='fastq')
print(reads)
seqreads = list(reads)
print(seqreads)

[SeqRecord(seq=Seq('GCTTAAGTTATTTAGTGCGGCCGCCTATGGTGCACTATTATTTATCTATCGTGA...AGA'), id='HISEQ:621:HMJGNBCX2:1:1101:1797:2150', name='HISEQ:621:HMJGNBCX2:1:1101:1797:2150', description='HISEQ:621:HMJGNBCX2:1:1101:1797:2150 1:N:0:ATGTCA', dbxrefs=[]), SeqRecord(seq=Seq('CTTCCTGGTCACGGTTGCGGCCGCCTATGGTGCATCATTATATGCAAATCCGGC...CGA'), id='HISEQ:621:HMJGNBCX2:1:1101:9239:2123', name='HISEQ:621:HMJGNBCX2:1:1101:9239:2123', description='HISEQ:621:HMJGNBCX2:1:1101:9239:2123 1:N:0:ATGTCA', dbxrefs=[]), SeqRecord(seq=Seq('CACGGCTATTACCCCGCGGCCGCCTATGGTGCACTATTATTTATCTATCGTGAA...CGA'), id='HISEQ:621:HMJGNBCX2:1:1101:11925:2202', name='HISEQ:621:HMJGNBCX2:1:1101:11925:2202', description='HISEQ:621:HMJGNBCX2:1:1101:11925:2202 1:N:0:ATGTCA', dbxrefs=[]), SeqRecord(seq=Seq('TCAGCGATGAATGTGGGCGGCCGCCTATGTTGCATCATTATATGCAAATCCGGC...TGA'), id='HISEQ:621:HMJGNBCX2:1:1101:14188:2196', name='HISEQ:621:HMJGNBCX2:1:1101:14188:2196', description='HISEQ:621:HMJGNBCX2:1:1101:14188:2196 1:N:0:ATGTCA', dbxrefs=[]

How many reads were there?

In [16]:
print(f"Found {len(seqreads)} sequencing reads.")
len(seqreads)

Found 10000 sequencing reads.


10000

#### Reads are read as `SeqRecord` objects
[SeqRecord](https://biopython.org/wiki/SeqRecord) objects have a lot of information, including the header, quality scores, etc.

Let's look at the first read:


In [17]:
seqreads[0]

SeqRecord(seq=Seq('GCTTAAGTTATTTAGTGCGGCCGCCTATGGTGCACTATTATTTATCTATCGTGA...AGA'), id='HISEQ:621:HMJGNBCX2:1:1101:1797:2150', name='HISEQ:621:HMJGNBCX2:1:1101:1797:2150', description='HISEQ:621:HMJGNBCX2:1:1101:1797:2150 1:N:0:ATGTCA', dbxrefs=[])

In [19]:
type(seqreads[0])

Bio.SeqRecord.SeqRecord


If you want to just access the sequence element of each `SeqRecord`, you can do this as follows:

In [20]:
seqreads[0].seq

Seq('GCTTAAGTTATTTAGTGCGGCCGCCTATGGTGCACTATTATTTATCTATCGTGA...AGA')

#### Sequences are `Seq` objects


In [21]:
type(seqreads[0].seq)

Bio.Seq.Seq

Let's make a list of just the sequences from our `seqreads`:

In [27]:
seqreads_Seq = []
for seqrecord in seqreads:
    sequence = seqrecord.seq # isolate the sequence from the seqrecord
    seqreads_Seq.append(sequence) # add string sequence to list

In [30]:
seqreads_Seq[0:5]

[Seq('GCTTAAGTTATTTAGTGCGGCCGCCTATGGTGCACTATTATTTATCTATCGTGA...AGA'),
 Seq('CTTCCTGGTCACGGTTGCGGCCGCCTATGGTGCATCATTATATGCAAATCCGGC...CGA'),
 Seq('CACGGCTATTACCCCGCGGCCGCCTATGGTGCACTATTATTTATCTATCGTGAA...CGA'),
 Seq('TCAGCGATGAATGTGGGCGGCCGCCTATGTTGCATCATTATATGCAAATCCGGC...TGA'),
 Seq('ATATGGGAGACGATAAGCGGCCGCCTATGGTGCATCATTATATGCAAATCCGGC...CTC')]

`Seq` objects come with many built-in methods specifically for working with sequences. 
We'll mostly be using sequences as `Seq` objects, but if you need to convert a `Seq` object to a regular `string` to use standard string methods, you can do so like this:

In [41]:
# let's isolate just the first sequence
seq = seqreads_Seq[0]
print(seq)

# convert this sequence to a string object
seq_string = str(seq)

print(seq_string)

GCTTAAGTTATTTAGTGCGGCCGCCTATGGTGCACTATTATTTATCTATCGTGAAAGGGAGTTCTGCTCCATCAGGCCAAGATCGGAAGAGCACACGTCTGAACTCCAGTCACATGTCAGAATCTCGTATGCCGTCTTCTGCTTGAAAAAAAAAAAAAAGCTGAAATTAATAATTTTGAAACCAGTTTTGTAAACGCAGCACTAAAATGAAGGCATGCAACGACGATGTTTATTGACACGGAATAGCAGA
GCTTAAGTTATTTAGTGCGGCCGCCTATGGTGCACTATTATTTATCTATCGTGAAAGGGAGTTCTGCTCCATCAGGCCAAGATCGGAAGAGCACACGTCTGAACTCCAGTCACATGTCAGAATCTCGTATGCCGTCTTCTGCTTGAAAAAAAAAAAAAAGCTGAAATTAATAATTTTGAAACCAGTTTTGTAAACGCAGCACTAAAATGAAGGCATGCAACGACGATGTTTATTGACACGGAATAGCAGA


#### Built-in `Seq` methods

`Biopython` has many useful built-in functions for working with sequencing data. 
We will discuss a few examples in class from the submodule [Bio.Seq](https://biopython.org/docs/1.75/api/Bio.Seq.html), but feel free to read about more [here](https://biopython.org/wiki/Seq)

We can use this module to get the __complement__ and __reverse complement__ of a sequence:

In [32]:
# recall we previously saved a sequence as the `seq` variable
seq

Seq('GCTTAAGTTATTTAGTGCGGCCGCCTATGGTGCACTATTATTTATCTATCGTGA...AGA')

In [34]:
seq.complement()

Seq('CGAATTCAATAAATCACGCCGGCGGATACCACGTGATAATAAATAGATAGCACT...TCT')

In [35]:
seq.reverse_complement()

Seq('TCTGCTATTCCGTGTCAATAAACATCGTCGTTGCATGCCTTCATTTTAGTGCTG...AGC')

We can use this module to __transcribe__ and __translate__ a sequence:

In [36]:
seq.transcribe()

Seq('GCUUAAGUUAUUUAGUGCGGCCGCCUAUGGUGCACUAUUAUUUAUCUAUCGUGA...AGA')

In [37]:
seq.translate()

/opt/conda/lib/python3.9/site-packages/Bio/Seq.py:2979: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


Seq('A*VI*CGRLWCTIIYLS*KGVLLHQAKIGRAHV*TPVTCQNLVCRLLLEKKKKA...E*Q')

__If you choose to use these methods, remember that a `Seq` object is not a string. You will need to convert your sequence back to a string before using methods/functions that require strings.__

---

## __A real biological analysis: parsing barcodes__
<a id='real_analysis'></a>
The reads that we just read as `seqreads_Seq` come from a real sequencing run of influenza virus HA and NA genes.

The __actual sequences__ are as follows:

    5'-[end of HA/NA]-AGGCGGCCGC-[16 X N barcode]-3'

    
The __sequencing run reads__ from the reverse end of the molecules (in 5'>3' orientation), so the sequencing reads are as follows:

    5'-[reverse complement of 16 X N barcode]-GCGGCCGCCT-[reverse complement of the end of HA/NA]-3'




We want to determine which reads have valid sequences, get the barcodes out of strings, and count the barcodes.
So this requires setting up an analysis that does the following:

 1. Get the reverse complement of each read.
 2. Determine if it matches the expected pattern (with the correct length barcode and constant sequence)
 3. If it matches, extract the barcode and add it to a dictionary to keep track of counts.

### __Group activity__
Work together to write some code to do this.
I have created a code chunk for each step (with some parts filled in). 
Remember to run the code chunks in the correct order!

For your homework, you will be asked to extend this in-class analysis to get statistics for HA and NA seperately.

In [42]:
# load necessary packages
import re
import Bio.SeqIO
import Bio.Seq

__Step 1:__ You'll need to write a function that identifies a barcode with a known upstream sequence. 
I've provided the documentation here--__try writing this function on your own.__

_Hint: we wrote a similar search pattern earlier_

_Hint 2: You can use the built-in reverse complement method_

_Hint 3: You will want to convert the sequence to a string before searching for regular expressions_

In [ ]:
def read_barcode(seqread, bclen, upstream='AGGCGGCCGC'):
    """Identify barcode with known upstream sequence.
    
    Parameters
    ----------
    seqread : Seq object
        Nucleotide sequence read matching UPSTREAM-BARCODE in reverse orientation.
    bclen : int
        Length of barcode
    upstream: str
        Sequence upstream of the barcode.
        
    Returns
    -------
    str or None
        Sequence of the barcode in the forward orientation, or `None` if no match to expected barcoded sequence.
        
    Example
    -------
    >>> read_barcode(Bio.Seq.Seq('TTTTTTTTTTTTTTTTGCGGCCGCCT'), bclen=16)
    'AAAAAAAAAAAAAAAA'
        
    """
    
    rev_comp = seqread.reverse_complement()
    
    barcode_search = re.compile(upstream + '(?P<barcode>[ATCG]{' + str(bclen) + '})')
    match = barcode_search.search(str(rev_comp))
    
    if match:
        barcode_match = match.group('barcode')
        return barcode_match 
    else:
        return None 

In [90]:
read_barcode(Bio.Seq.Seq('TTTTTTTTTTTTTTTTGCGGCCGCCT'), bclen=16)

'AAAAAAAAAAAAAAAA'

__Step 2:__ Read sequences from the barcodes_R1.fastq file and create a list of only the sequences (as Seq objects). __We already did this step earlier so you can move to step 3 after running this code chunk__

In [87]:
# run this code chunk...
seqreads = list(Bio.SeqIO.parse('barcodes_R1.fastq', 'fastq'))

seqreads_Seq = []
for seqrecord in seqreads:
    seqreads_Seq.append(seqrecord.seq)

print(seqreads_Seq[1:10])

[Seq('CTTCCTGGTCACGGTTGCGGCCGCCTATGGTGCATCATTATATGCAAATCCGGC...CGA'), Seq('CACGGCTATTACCCCGCGGCCGCCTATGGTGCACTATTATTTATCTATCGTGAA...CGA'), Seq('TCAGCGATGAATGTGGGCGGCCGCCTATGTTGCATCATTATATGCAAATCCGGC...TGA'), Seq('ATATGGGAGACGATAAGCGGCCGCCTATGGTGCATCATTATATGCAAATCCGGC...CTC'), Seq('GTTGGGAAACGGGTAGGCGGCCGCCTATGGTGCACTATTATTTATCTATCGTGA...TTA'), Seq('AGGGATGACTGGTATGGCGGCCGCCTATGGTGCATCATTATATGCAAATCCGGC...TGG'), Seq('TGACTTATACGTAAGTGCGGCCGCCTATGGTGCATCATTATATGCAAATCCGGC...CGA'), Seq('ATGGTATAGTAGTAGCGCGGCCGCCTATGGTGCATCATTATATGCAAATCCGGC...TGA'), Seq('CACGTCTATTACCCCGCGGCCGCCTATGGTGCACTATTATTTATCTATCGTGAA...AAA')]


__Step 3:__ Get the counts of all barcodes. _(Hint: you might want to store barcodes and counts in a dictionary, and also keep track of the number of sequences that don't have a valid barcode)_

Please name your dictionary `barcode_counts`

In [108]:
# your code here ...

def count_barcodes(seqs, bclen=16, upstream='AGGCGGCCGC'):
    """Parse and count barcodes.
    
    Parameters
    ----------
    seqs :  list
        List of Seq objects (DNA sequences).
    bclen : int
        Length of barcode
    upstream : str
        Sequence upstream of barcode.
        
    Returns
    -------
    dict
        Keyed by each valid barcode, value is number of times the barcode
        is observed.
        
    Note
    ----
    The function is **not** case-sensitive, and all barcodes are reported
    in upper-case.
    
    """
    
    barcode_counts = {'nonvalid':0}
    
    for seq in seqs:

        rev_comp = seq.reverse_complement()

        barcode_search = re.compile(upstream + '(?P<barcode>[ATCG]{' + str(bclen) + '})')
        match = barcode_search.search(str(rev_comp))
        
        if match:
            barcode_match = match.group('barcode')
            
            if barcode_match in barcode_counts:
                barcode_counts[barcode_match] += 1
            else:
                barcode_counts[barcode_match] = 1
        else:
            barcode_counts['nonvalid'] += 1
        
    return(barcode_counts)


__Step 4:__ Report the total number of sequences parsed, and how many lacked a valid barcode.

In [109]:
# your code here ...
barcode_counts = count_barcodes(seqreads_Seq)
print('There were ' + str(len(seqreads_Seq)) + ' sequences parsed.')
print('There were ' + str(barcode_counts['nonvalid']) + ' sequences lacking a valid barcode.')


There were 10000 sequences parsed.
There were 563 sequences lacking a valid barcode.
